### 1. Iz Annotations/train.json filtrirati samo ona pitanja koja ne zahtevaju prepoznavanje teksta (izlazi van domena rešenja)
### 2. Podeliti filtrirani skup na train i test deo (data/train.json i data/test.json)

### Napomena 1 - Annotations/val.json i Annotations/test.json necemo koristiti ni za obucavanje ni za evaluaciju zato sto nemamo informacije o tome da li zahtevaju prepoznavanje teksta
### Napomena 2 - Ravnomerna podela na osnovu answer_type (number, yes/no, other, unanswerable)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [118]:
ocr = pd.read_csv('data/ocr.csv')
ocr.head()

,IMG,QSN,TXT,OBJ,COL,CNT,OTH
0,VizWiz_train_00000000.jpg,What's the name of this product?,4,3,0,0,0
1,VizWiz_train_00000001.jpg,Can you tell me what is in this can please?,2,5,1,1,0
2,VizWiz_train_00000002.jpg,Is this enchilada sauce or is this tomatoes? ...,4,3,1,0,0
3,VizWiz_train_00000003.jpg,What is the captcha on this screenshot?,4,2,1,1,0
4,VizWiz_train_00000004.jpg,What is this item?,3,5,1,0,0


In [119]:
ocr.TXT.describe()

count    14239.000000
mean         2.107732
std          1.819784
min          0.000000
25%          0.000000
50%          2.000000
75%          4.000000
max          5.000000
Name: TXT, dtype: float64

In [120]:
ocr_filtered = ocr[ocr.TXT <= 1]
ocr_filtered.head()

,IMG,QSN,TXT,OBJ,COL,CNT,OTH
5,VizWiz_train_00000005.jpg,What's this?,1,5,1,0,0
8,VizWiz_train_00000008.jpg,What color do these look?,1,3,4,0,0
9,VizWiz_train_00000009.jpg,Surface look clean? Thank you.,0,5,1,0,0
10,VizWiz_train_00000010.jpg,Is this.,1,5,2,0,0
11,VizWiz_train_00000012.jpg,what is this?,0,5,2,0,0


In [121]:
ocr_filtered.TXT.describe()

count    6241.000000
mean        0.273514
std         0.445798
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: TXT, dtype: float64

In [122]:
data = pd.read_json('Annotations/train.json')

In [123]:
data_filtered = data[data.image.isin(ocr_filtered.IMG)]
data_filtered.head()

,image,question,answers,answer_type,answerable
5,VizWiz_train_00000005.jpg,What's this?,"[{'answer_confidence': 'yes', 'answer': 'boots...",other,1
8,VizWiz_train_00000008.jpg,What color do these look?,"[{'answer_confidence': 'yes', 'answer': 'orang...",other,1
9,VizWiz_train_00000009.jpg,Surface look clean? Thank you.,"[{'answer_confidence': 'yes', 'answer': 'yes'}...",yes/no,1
10,VizWiz_train_00000010.jpg,Is this.,"[{'answer_confidence': 'maybe', 'answer': 'and...",other,1
12,VizWiz_train_00000012.jpg,what is this?,"[{'answer_confidence': 'yes', 'answer': 'remot...",other,1


In [124]:
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6238 entries, 5 to 19924
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        6238 non-null   object
 1   question     6238 non-null   object
 2   answers      6238 non-null   object
 3   answer_type  6238 non-null   object
 4   answerable   6238 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 292.4+ KB


### Zašto ocr_filtered ima 6241, a data_filtered 6238 elementara? nzm

In [125]:
data_filtered.answer_type.describe()

count      6238
unique        4
top       other
freq       5678
Name: answer_type, dtype: object

In [126]:
data_filtered.answer_type.unique()

array(['other', 'yes/no', 'number', 'unanswerable'], dtype=object)

In [127]:
train, test = train_test_split(data_filtered, train_size=0.9, stratify=data_filtered.answer_type)
train.to_json('data/train.json')
test.to_json('data/test.json')

In [128]:
train = pd.read_json('data/train.json')
train.head()

,image,question,answers,answer_type,answerable
1616,VizWiz_train_00001616.jpg,What is it?,"[{'answer_confidence': 'yes', 'answer': 'rug'}...",other,1
11910,VizWiz_train_00011910.jpg,WHAT DO YOU SEE?,"[{'answer_confidence': 'maybe', 'answer': 'leg...",other,1
13392,VizWiz_train_00013392.jpg,What is this?,"[{'answer_confidence': 'no', 'answer': 'pendan...",other,1
7274,VizWiz_train_00007274.jpg,What do I see,"[{'answer': 'toilet paper', 'answer_confidence...",other,1
15488,VizWiz_train_00015488.jpg,What color are these socks?,"[{'answer_confidence': 'yes', 'answer': 'black...",other,1


In [129]:
test = pd.read_json('data/test.json')
test.head()

,image,question,answers,answer_type,answerable
7824,VizWiz_train_00007824.jpg,What color is this?,"[{'answer_confidence': 'maybe', 'answer': 'blu...",other,1
7288,VizWiz_train_00007288.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'cream...",other,1
6489,VizWiz_train_00006489.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'mug'}...",other,1
9376,VizWiz_train_00009376.jpg,what color is the shirt,"[{'answer_confidence': 'yes', 'answer': 'white...",other,1
6591,VizWiz_train_00006591.jpg,What are these pills?,"[{'answer_confidence': 'yes', 'answer': 'wefwe...",other,1


In [130]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5614 entries, 1616 to 4966
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        5614 non-null   object
 1   question     5614 non-null   object
 2   answers      5614 non-null   object
 3   answer_type  5614 non-null   object
 4   answerable   5614 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 263.2+ KB


In [131]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 7824 to 11965
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        624 non-null    object
 1   question     624 non-null    object
 2   answers      624 non-null    object
 3   answer_type  624 non-null    object
 4   answerable   624 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 29.2+ KB


## Fomiranje finalnog dataset-a

In [2]:
dataset = 'test'

In [3]:
qdf = pd.read_csv('data/' + dataset + '_questions.csv', index_col=0)
qdf.head()

,image,question
0,VizWiz_train_00007824.jpg,0.11439254134893417|0.3182021379470825|-0.1863...
1,VizWiz_train_00007288.jpg,-0.11341997981071472|0.2821967899799347|-0.144...
2,VizWiz_train_00006489.jpg,-0.11341997981071472|0.2821967899799347|-0.144...
3,VizWiz_train_00009376.jpg,-0.021189622581005096|0.12264803051948547|-0.3...
4,VizWiz_train_00006591.jpg,-0.041424460709095|0.5233025550842285|0.043939...


In [4]:
qdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 623
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     624 non-null    object
 1   question  624 non-null    object
dtypes: object(2)
memory usage: 14.6+ KB


In [5]:
rdf = pd.read_csv('data/' + dataset + '_regions.csv', index_col=0)
rdf.head()

,image,vectors
0,VizWiz_train_00007824.jpg,0.19898295402526855|0.1904696673154831|0.34966...
1,VizWiz_train_00007288.jpg,0.5149837136268616|1.2104671001434326|1.168068...
2,VizWiz_train_00006489.jpg,0.28766393661499023|0.45064228773117065|0.5715...
4,VizWiz_train_00006591.jpg,0.6279891729354858|0.39521658420562744|0.40628...
5,VizWiz_train_00004660.jpg,0.7742525935173035|0.42165425419807434|0.54338...


In [6]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 180
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    624 non-null    object
 1   vectors  624 non-null    object
dtypes: object(2)
memory usage: 14.6+ KB


In [7]:
adf = pd.read_csv('data/' + dataset + '_answers.csv')
adf.head()

,0,0.1,00 310 539,1,1 08 11,1 3,1 4 ounce,1 cup,1 dollar,1 dollar bill,...,yes green grey,yes grey shirt,yes no,yes on,yes white,yes yes,yogurt,yoplait,yorkie,zipper
0,7824,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7288,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6489,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9376,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6591,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
adf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Columns: 2328 entries, 0 to zipper
dtypes: int64(2328)
memory usage: 11.1 MB


In [9]:
adf['0'].values

array([ 7824,  7288,  6489,  9376,  6591,  4660, 17701, 12220, 16226,
        4732, 18406,   689,  9120,  1803, 15363, 17918, 12499,  8674,
        4471, 15211,  4330,  5895,  8997,  4607, 10753,  1238,  7575,
        5213, 18038,  8381,  9437,  1035,  9413, 15875,  6545, 10016,
       15557, 13868,  5822,  7852,  6954, 13705, 16929,  5658, 17366,
        9579, 19665,  6110, 13032,  3024, 12067, 18484,  4468, 13431,
       10958, 16988, 19694,  5243,  2262, 11439,  6697,  1891, 12689,
        5936,  7772, 17036,  7884, 14669,  4482, 11613, 14314, 12025,
       13467,  1140,  9797, 15865,  5790, 16851,  5401, 18646, 10599,
        9041,  2037, 11106, 17885,  6352,  3258, 14603, 19731, 17828,
        9221,  6948, 13864, 10577, 13940,  8497, 11200, 14768,   678,
       16329, 14852, 15729,  6332, 12652,  2224, 13092,  4096,  9680,
        4133,  6138, 19526,  1971,  8331, 14954,  4369, 13081,  7938,
       10939,  7292,  4159,   832, 19475, 17289,  9690, 10294, 12472,
       15848, 13743,

In [10]:
atdf = pd.read_json('data/' + dataset + '.json')
atdf.head()

,image,question,answers,answer_type,answerable
7824,VizWiz_train_00007824.jpg,What color is this?,"[{'answer_confidence': 'maybe', 'answer': 'blu...",other,1
7288,VizWiz_train_00007288.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'cream...",other,1
6489,VizWiz_train_00006489.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'mug'}...",other,1
9376,VizWiz_train_00009376.jpg,what color is the shirt,"[{'answer_confidence': 'yes', 'answer': 'white...",other,1
6591,VizWiz_train_00006591.jpg,What are these pills?,"[{'answer_confidence': 'yes', 'answer': 'wefwe...",other,1


In [11]:
def add_id_column(df):
    id_col = [get_id(image) for image in df.image]
    df['id'] = id_col
    return df
    
def get_id(image):
    return int(image.removeprefix('VizWiz_train_').removesuffix('.jpg'))

In [12]:
qdf = add_id_column(qdf)
qdf.drop(columns=['image'], inplace=True)
qdf.head()

,question,id
0,0.11439254134893417|0.3182021379470825|-0.1863...,7824
1,-0.11341997981071472|0.2821967899799347|-0.144...,7288
2,-0.11341997981071472|0.2821967899799347|-0.144...,6489
3,-0.021189622581005096|0.12264803051948547|-0.3...,9376
4,-0.041424460709095|0.5233025550842285|0.043939...,6591


In [13]:
rdf.rename(columns = {'vectors':'img_regions'}, inplace = True)
rdf = add_id_column(rdf)
rdf.drop(columns=['image'], inplace = True)
rdf.head()

,img_regions,id
0,0.19898295402526855|0.1904696673154831|0.34966...,7824
1,0.5149837136268616|1.2104671001434326|1.168068...,7288
2,0.28766393661499023|0.45064228773117065|0.5715...,6489
4,0.6279891729354858|0.39521658420562744|0.40628...,6591
5,0.7742525935173035|0.42165425419807434|0.54338...,4660


In [14]:
adf.rename(columns = {'0':'id'}, inplace = True)
adf.head()

,id,0.1,00 310 539,1,1 08 11,1 3,1 4 ounce,1 cup,1 dollar,1 dollar bill,...,yes green grey,yes grey shirt,yes no,yes on,yes white,yes yes,yogurt,yoplait,yorkie,zipper
0,7824,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7288,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6489,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9376,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6591,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
adf.id.values

array([ 7824,  7288,  6489,  9376,  6591,  4660, 17701, 12220, 16226,
        4732, 18406,   689,  9120,  1803, 15363, 17918, 12499,  8674,
        4471, 15211,  4330,  5895,  8997,  4607, 10753,  1238,  7575,
        5213, 18038,  8381,  9437,  1035,  9413, 15875,  6545, 10016,
       15557, 13868,  5822,  7852,  6954, 13705, 16929,  5658, 17366,
        9579, 19665,  6110, 13032,  3024, 12067, 18484,  4468, 13431,
       10958, 16988, 19694,  5243,  2262, 11439,  6697,  1891, 12689,
        5936,  7772, 17036,  7884, 14669,  4482, 11613, 14314, 12025,
       13467,  1140,  9797, 15865,  5790, 16851,  5401, 18646, 10599,
        9041,  2037, 11106, 17885,  6352,  3258, 14603, 19731, 17828,
        9221,  6948, 13864, 10577, 13940,  8497, 11200, 14768,   678,
       16329, 14852, 15729,  6332, 12652,  2224, 13092,  4096,  9680,
        4133,  6138, 19526,  1971,  8331, 14954,  4369, 13081,  7938,
       10939,  7292,  4159,   832, 19475, 17289,  9690, 10294, 12472,
       15848, 13743,

In [16]:
atdf = add_id_column(atdf)
atdf.drop(columns=['image', 'question', 'answers', 'answerable'], inplace = True)
atdf.head()

,answer_type,id
7824,other,7824
7288,other,7288
6489,other,6489
9376,other,9376
6591,other,6591


In [17]:
atdf.id.values

array([ 7824,  7288,  6489,  9376,  6591,  4660, 17701, 12220, 16226,
        4732, 18406,   689,  9120,  1803, 15363, 17918, 12499,  8674,
        4471, 15211,  4330,  5895,  8997,  4607, 10753,  1238,  7575,
        5213, 18038,  8381,  9437,  1035,  9413, 15875,  6545, 10016,
       15557, 13868,  5822,  7852,  6954, 13705, 16929,  5658, 17366,
        9579, 19665,  6110, 13032,  3024, 12067, 18484,  4468, 13431,
       10958, 16988, 19694,  5243,  2262, 11439,  6697,  1891, 12689,
        5936,  7772, 17036,  7884, 14669,  4482, 11613, 14314, 12025,
       13467,  1140,  9797, 15865,  5790, 16851,  5401, 18646, 10599,
        9041,  2037, 11106, 17885,  6352,  3258, 14603, 19731, 17828,
        9221,  6948, 13864, 10577, 13940,  8497, 11200, 14768,   678,
       16329, 14852, 15729,  6332, 12652,  2224, 13092,  4096,  9680,
        4133,  6138, 19526,  1971,  8331, 14954,  4369, 13081,  7938,
       10939,  7292,  4159,   832, 19475, 17289,  9690, 10294, 12472,
       15848, 13743,

In [19]:
df = qdf.merge(rdf, on = 'id')
df = df.merge(atdf, on = 'id')
df = df.merge(adf, on = 'id')
df.head(20)

,question,id,img_regions,answer_type,0.1,00 310 539,1,1 08 11,1 3,1 4 ounce,...,yes green grey,yes grey shirt,yes no,yes on,yes white,yes yes,yogurt,yoplait,yorkie,zipper
0,0.11439254134893417|0.3182021379470825|-0.1863...,7824,0.19898295402526855|0.1904696673154831|0.34966...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.11341997981071472|0.2821967899799347|-0.144...,7288,0.5149837136268616|1.2104671001434326|1.168068...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.11341997981071472|0.2821967899799347|-0.144...,6489,0.28766393661499023|0.45064228773117065|0.5715...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.021189622581005096|0.12264803051948547|-0.3...,9376,1.631096363067627|2.0074942111968994|1.7980965...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.041424460709095|0.5233025550842285|0.043939...,6591,0.6279891729354858|0.39521658420562744|0.40628...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,-0.11341997981071472|0.2821967899799347|-0.144...,4660,0.7742525935173035|0.42165425419807434|0.54338...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0.011887861415743828|0.2800214886665344|-0.191...,17701,0.630643904209137|0.2332589477300644|1.2390974...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,-0.2558656334877014|-0.13916736841201782|0.024...,12220,2.7533702850341797|0.07866501063108444|0.71917...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,-0.11341997981071472|0.2821967899799347|-0.144...,16226,1.9706660509109497|0.7900238633155823|3.356488...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0.11439254134893417|0.3182021379470825|-0.1863...,4732,0.22546489536762238|0.007291094865649939|0.072...,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 623
Columns: 2331 entries, question to zipper
dtypes: int64(2328), object(3)
memory usage: 11.1+ MB


In [21]:
df.id.values

array([ 7824,  7288,  6489,  9376,  6591,  4660, 17701, 12220, 16226,
        4732, 18406,   689,  9120,  1803, 15363, 17918, 12499,  8674,
        4471, 15211,  4330,  5895,  8997,  4607, 10753,  1238,  7575,
        5213, 18038,  8381,  9437,  1035,  9413, 15875,  6545, 10016,
       15557, 13868,  5822,  7852,  6954, 13705, 16929,  5658, 17366,
        9579, 19665,  6110, 13032,  3024, 12067, 18484,  4468, 13431,
       10958, 16988, 19694,  5243,  2262, 11439,  6697,  1891, 12689,
        5936,  7772, 17036,  7884, 14669,  4482, 11613, 14314, 12025,
       13467,  1140,  9797, 15865,  5790, 16851,  5401, 18646, 10599,
        9041,  2037, 11106, 17885,  6352,  3258, 14603, 19731, 17828,
        9221,  6948, 13864, 10577, 13940,  8497, 11200, 14768,   678,
       16329, 14852, 15729,  6332, 12652,  2224, 13092,  4096,  9680,
        4133,  6138, 19526,  1971,  8331, 14954,  4369, 13081,  7938,
       10939,  7292,  4159,   832, 19475, 17289,  9690, 10294, 12472,
       15848, 13743,

In [23]:
df.to_csv('data/' + dataset + '.csv', index=False)

In [24]:
df = pd.read_csv('data/' + dataset + '.csv')
df.id.values

array([ 7824,  7288,  6489,  9376,  6591,  4660, 17701, 12220, 16226,
        4732, 18406,   689,  9120,  1803, 15363, 17918, 12499,  8674,
        4471, 15211,  4330,  5895,  8997,  4607, 10753,  1238,  7575,
        5213, 18038,  8381,  9437,  1035,  9413, 15875,  6545, 10016,
       15557, 13868,  5822,  7852,  6954, 13705, 16929,  5658, 17366,
        9579, 19665,  6110, 13032,  3024, 12067, 18484,  4468, 13431,
       10958, 16988, 19694,  5243,  2262, 11439,  6697,  1891, 12689,
        5936,  7772, 17036,  7884, 14669,  4482, 11613, 14314, 12025,
       13467,  1140,  9797, 15865,  5790, 16851,  5401, 18646, 10599,
        9041,  2037, 11106, 17885,  6352,  3258, 14603, 19731, 17828,
        9221,  6948, 13864, 10577, 13940,  8497, 11200, 14768,   678,
       16329, 14852, 15729,  6332, 12652,  2224, 13092,  4096,  9680,
        4133,  6138, 19526,  1971,  8331, 14954,  4369, 13081,  7938,
       10939,  7292,  4159,   832, 19475, 17289,  9690, 10294, 12472,
       15848, 13743,

In [25]:
len(df.id.values)

624

In [27]:
df.id.nunique()

624